In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Lecture des données

In [6]:
transactions_101_104 = pd.read_csv('../raw_data/transaction_train_week_101_104.csv')
#101= dernière semaine d'aout
#102= première semaine septembre
#103= deuxième semaine septembre
#104= troisème semaine de septembre (qui nous servira de target dans le cadre du projet)

In [58]:
transactions_101_104.head()

,Unnamed: 0,Unnamed: 0.1,t_dat,customer_id,article_id,price,sales_channel_id
0,0,30665422,101,0000e9a4db2da4e9c68558ad1e7df570d02767b213ec6b...,880553001,0.025407,2
1,1,30665423,101,0000e9a4db2da4e9c68558ad1e7df570d02767b213ec6b...,880553001,0.025407,2
2,2,30665424,101,00047d50bde8751b26aaec7536ccc5fd92828ffa0a396d...,909081004,0.033881,2
3,3,30665425,101,00047d50bde8751b26aaec7536ccc5fd92828ffa0a396d...,516903024,0.025407,2
4,4,30665426,101,00047d50bde8751b26aaec7536ccc5fd92828ffa0a396d...,897679003,0.042356,2


In [7]:
# note du 06042022: à ce stade de l'analyse les customers sans information sur l'âge ne sont pas inclus
S1= pd.read_csv('../raw_data/customers_age_16_17.csv')
S2= pd.read_csv('../raw_data/customers_age_18_19.csv')
S3= pd.read_csv('../raw_data/customers_age_20_21.csv')
S4= pd.read_csv('../raw_data/customers_age_22_23.csv')
S5= pd.read_csv('../raw_data/customers_age_24_25.csv')
S6= pd.read_csv('../raw_data/customers_age_26_27.csv')
S7= pd.read_csv('../raw_data/customers_age_28_29.csv')
S8= pd.read_csv('../raw_data/customers_age_30_34.csv')
S9= pd.read_csv('../raw_data/customers_age_35_39.csv')
S10= pd.read_csv('../raw_data/customers_age_40_44.csv')
S11= pd.read_csv('../raw_data/customers_age_45_49.csv')
S12= pd.read_csv('../raw_data/customers_age_50_54.csv')
S13= pd.read_csv('../raw_data/customers_age_55_59.csv')
S14= pd.read_csv('../raw_data/customers_age_60.csv')

In [8]:
S1.shape, S2.shape, S3.shape, S4.shape, S5.shape, S6.shape, S7.shape, S8.shape, S9.shape, S10.shape, S11.shape, S12.shape, S13.shape, S14.shape

((9553, 8),
 (62030, 8),
 (122726, 8),
 (106736, 8),
 (111113, 8),
 (102792, 8),
 (84991, 8),
 (150520, 8),
 (83548, 8),
 (85493, 8),
 (118625, 8),
 (135326, 8),
 (90916, 8),
 (91750, 8))

In [83]:
S1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9553 entries, 0 to 9552
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_id  9553 non-null   object
 1   age_id       9553 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 149.4+ KB


# Préparation du dataset Transactions

In [9]:
# je garde uniquement la week, le customer id pour le merge, et l'article ID pour récupérer les plus populaires:
transactions_101_104=transactions_101_104.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'price', 'sales_channel_id'])

In [10]:
transactions_101_104.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1057905 entries, 0 to 1057904
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   t_dat        1057905 non-null  int64 
 1   customer_id  1057905 non-null  object
 2   article_id   1057905 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 24.2+ MB


In [11]:
# je crée des dummies par week si jamais plus tard on a besoin de baselines pour d'autres semaines que la 104
article_history=transactions_101_104
article_history['in101']=article_history['t_dat'].apply(lambda x: 1 if x==101 else 0)
article_history['in102']=article_history['t_dat'].apply(lambda x: 1 if x==102 else 0)
article_history['in103']=article_history['t_dat'].apply(lambda x: 1 if x==103 else 0)
article_history['in104']=article_history['t_dat'].apply(lambda x: 1 if x==104 else 0)
article_history

,t_dat,customer_id,article_id,in101,in102,in103,in104
0,101,0000e9a4db2da4e9c68558ad1e7df570d02767b213ec6b...,880553001,1,0,0,0
1,101,0000e9a4db2da4e9c68558ad1e7df570d02767b213ec6b...,880553001,1,0,0,0
2,101,00047d50bde8751b26aaec7536ccc5fd92828ffa0a396d...,909081004,1,0,0,0
3,101,00047d50bde8751b26aaec7536ccc5fd92828ffa0a396d...,516903024,1,0,0,0
4,101,00047d50bde8751b26aaec7536ccc5fd92828ffa0a396d...,897679003,1,0,0,0
...,...,...,...,...,...,...,...
1057900,104,fff4381593e170ca0aea188998487c830d9a4070c9ec4b...,799365001,0,0,0,1
1057901,104,fff7e7674509592818bf453391af43a85eaaac9a52d858...,624486049,0,0,0,1
1057902,104,fff871bf24b40fd1290215414d760afaa69bb164d2b970...,717490010,0,0,0,1
1057903,104,fff871bf24b40fd1290215414d760afaa69bb164d2b970...,717490058,0,0,0,1


# merge des données segments aux données transactions

In [92]:
S1_achats=S1[['customer_id','age_id']].merge(article_history)

In [93]:
S1_achats.shape #(5955, 8)

(5955, 8)

In [95]:
S1_achats.head()

,customer_id,age_id,t_dat,article_id,in101,in102,in103,in104
0,0006ca2c5ecdbdcc1de5e72adffc6c33448fc2aaa62afe...,0,101,572998001,1,0,0,0
1,0006ca2c5ecdbdcc1de5e72adffc6c33448fc2aaa62afe...,0,101,572998009,1,0,0,0
2,00089f13f465ec902e5c49a3bb408c5e31205096d6f267...,0,101,926502001,1,0,0,0
3,000e3f587242eb077685a487ad27dad632a4801576dfd1...,0,101,501620049,1,0,0,0
4,001c42459c8978d55a69fa1528975893b011a6539631f3...,0,101,885077003,1,0,0,0


In [96]:
S1_articles=S1_achats.groupby(['article_id']).sum()
S1_articles.shape

(2501, 6)

In [103]:
S1_Top12_104=S1_articles.sort_values(by='in104', ascending=False).head(12)
S1_Top12_104

,age_id,t_dat,in101,in102,in103,in104
article_id,,,,,,
448509014,0,3723,4,2,5,25
797988002,0,3303,7,2,0,23
685814003,0,3725,1,0,16,19
715624052,0,2173,3,0,2,16
682550002,0,2282,0,0,6,16
456163086,0,2380,2,1,4,16
730683050,0,2269,5,2,0,15
715624001,0,3494,10,2,8,14
871517008,0,2171,3,0,4,14


In [58]:
def Baseline_0 (segment,transact_history,week): # exemple: week104name ='in104' dans l'historique des transactions
    
    week_dummy="in" + f'{week}'
    filtre_week=transact_history[week_dummy]==1
    achats=segment[['customer_id']].merge(transact_history[filtre_week][['customer_id','article_id',week_dummy]])
    articles= achats.groupby(['article_id']).sum()
    Top12_arr=articles.sort_values(by=week_dummy, ascending=False).head(12)
    Top12=""
    for i in range (0, 12):
        if i==0 :
            Top12=str(Top12_arr.index[i])
        else: 
            Top12=Top12 + " "+ str(Top12_arr.index[i])
            
    #je donne à tous mes customers de mon segment le baseline top12 calculée ci-dessus
    segment['Baseline0']=Top12
    
    #Ancien code ici *******************************************************************
    #je retourne sur leurs achats et je vérifie si l'articleID acheté est dans la TOP12 (1 si oui, O sinon)
    #achats['inBaseline0']=achats['article_id'].apply(lambda x: 1 if str(x) in Top12 else 0)
    #Baseline0_precision=achats.groupby(['customer_id']).count().drop(columns= ['article_id',week_dummy] ) 
    
    #penser à faire un changement de nom de col pour la précision
    #segment_BS=segment.merge(Baseline0_precision, how = 'left', on='customer_id') 
    #  *******************************************************************
    
    #je retourne sur leurs achats et je vérifie si l'articleID acheté est dans la TOP12 (1 si oui, O sinon)
    achats['inBaseline0']=achats['article_id'].apply(lambda x: 1 if str(x) in Top12 else 0)
    Baseline0_precision=achats.groupby(['customer_id']).count().drop(columns= ['article_id',week_dummy]).rename(columns={'inBaseline0':'Precision'})
    print("Statistiques concernant le segment:")
    print("Taille du segment : " + f'{len(segment)}')
    print("Nbre d'acheteurs : " + f'{Baseline0_precision.count()}')
    print("Nbre de ref_achetées : " + f'{articles.count()}')
    print("Précision moyenne:" + f'{Baseline0_precision.mean()}')
    
    segment_BS=segment.merge(Baseline0_precision, how = 'left', on='customer_id') 
    #return segment_BS
    return achats
  

In [57]:
Baseline_0(S1,article_history,'104')

Statistiques concernant le segment:
Taille du segment : 9553
Nbre d'acheteurs : Precision    1213
dtype: int64
Nbre de ref_achetées : in104    1663
dtype: int64
Précision moyenne:Precision    2.656224
dtype: float64


,customer_id,article_id,in104,inBaseline0
0,002921dfdcb8be4426c555a01cb8a009003db8c7582453...,778064003,1,0
1,002921dfdcb8be4426c555a01cb8a009003db8c7582453...,889661001,1,0
2,002921dfdcb8be4426c555a01cb8a009003db8c7582453...,800245011,1,0
3,002921dfdcb8be4426c555a01cb8a009003db8c7582453...,156231001,1,0
4,002921dfdcb8be4426c555a01cb8a009003db8c7582453...,156231001,1,0
...,...,...,...,...
3217,ffd6bb6d6a33e415467e69d18d28ec9e64061a4f58d67c...,914441006,1,0
3218,ffd6bb6d6a33e415467e69d18d28ec9e64061a4f58d67c...,914441004,1,0
3219,ffec95723bd3cfc50cef76f198716d18e395261f7cd8b1...,858147006,1,0
3220,ffec95723bd3cfc50cef76f198716d18e395261f7cd8b1...,892643001,1,0


In [41]:
Baseline_0(S2,article_history,'104')

Statistiques concernant le segment:
Taille du segment : 62030
Nbre d'acheteurs : Precision    6069
dtype: int64
Nbre de ref_achetées : in104    5079
dtype: int64
Précision moyenne:Precision    2.788103
dtype: float64


,customer_id,article_id,in104,inBaseline0
0,0006bfcec82c6c132276c0a5549ae13645c9ab77b0243f...,882354001,1,0
1,0006bfcec82c6c132276c0a5549ae13645c9ab77b0243f...,817354001,1,0
2,0006bfcec82c6c132276c0a5549ae13645c9ab77b0243f...,817354001,1,0
3,0006bfcec82c6c132276c0a5549ae13645c9ab77b0243f...,903473001,1,0
4,0006bfcec82c6c132276c0a5549ae13645c9ab77b0243f...,881577002,1,0
...,...,...,...,...
16916,fff673307d4cdbf688e4a0bcfe7f671036033dbe7eba01...,860525009,1,0
16917,fff673307d4cdbf688e4a0bcfe7f671036033dbe7eba01...,759871003,1,0
16918,fff673307d4cdbf688e4a0bcfe7f671036033dbe7eba01...,456163060,1,0
16919,fff673307d4cdbf688e4a0bcfe7f671036033dbe7eba01...,711053003,1,0


In [42]:
Baseline_0(S3,article_history,'104')

Statistiques concernant le segment:
Taille du segment : 122726
Nbre d'acheteurs : Precision    13203
dtype: int64
Nbre de ref_achetées : in104    8113
dtype: int64
Précision moyenne:Precision    3.102401
dtype: float64


,customer_id,article_id,in104,inBaseline0
0,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,719530003,1,0
1,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,448509014,1,1
2,0001f8cef6b9702d54abf66fd89eb21014bf98567065a9...,820671001,1,0
3,0005340aa69bb5a28d98712a36d8f669024bce137e3c82...,821395014,1,0
4,0005340aa69bb5a28d98712a36d8f669024bce137e3c82...,759814036,1,0
...,...,...,...,...
40956,fffd0248a95c2e49fee876ff93598e2e20839e51b9b767...,926745002,1,0
40957,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,689365050,1,0
40958,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,884081001,1,0
40959,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,794819001,1,0


In [43]:
Baseline_0(S4,article_history,'104')

Statistiques concernant le segment:
Taille du segment : 106736
Nbre d'acheteurs : Precision    13091
dtype: int64
Nbre de ref_achetées : in104    9143
dtype: int64
Précision moyenne:Precision    3.688794
dtype: float64


,customer_id,article_id,in104,inBaseline0
0,00077dbd5c4a4991e092e63893ccf29294a9d5c46e8501...,903762001,1,0
1,00077dbd5c4a4991e092e63893ccf29294a9d5c46e8501...,879189005,1,0
2,0007c93b2a5ff71d3358d0ee4391c47bbf6b8e7d4d2bf6...,903773001,1,0
3,0007c93b2a5ff71d3358d0ee4391c47bbf6b8e7d4d2bf6...,556539010,1,0
4,0007c93b2a5ff71d3358d0ee4391c47bbf6b8e7d4d2bf6...,889387002,1,0
...,...,...,...,...
48285,fff3573d9131d15da6a46c1ca8f03b5d37e4f6b804171e...,876204002,1,0
48286,fff3573d9131d15da6a46c1ca8f03b5d37e4f6b804171e...,888715001,1,0
48287,fff3573d9131d15da6a46c1ca8f03b5d37e4f6b804171e...,866503005,1,0
48288,fff3573d9131d15da6a46c1ca8f03b5d37e4f6b804171e...,916796006,1,0


In [44]:
Baseline_0(S5,article_history,'104')

Statistiques concernant le segment:
Taille du segment : 111113
Nbre d'acheteurs : Precision    14092
dtype: int64
Nbre de ref_achetées : in104    10282
dtype: int64
Précision moyenne:Precision    4.037184
dtype: float64


,customer_id,article_id,in104,inBaseline0
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,794321007,1,0
1,0010e8eb18f131e724d6997909af0808adbba057529edb...,237347060,1,0
2,0010e8eb18f131e724d6997909af0808adbba057529edb...,562245001,1,0
3,0010e8eb18f131e724d6997909af0808adbba057529edb...,610776105,1,0
4,0010e8eb18f131e724d6997909af0808adbba057529edb...,456163060,1,0
...,...,...,...,...
56887,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,720125039,1,0
56888,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,740922009,1,0
56889,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,791587007,1,0
56890,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,804992033,1,0


In [45]:
Baseline_0(S6,article_history,'104')

Statistiques concernant le segment:
Taille du segment : 102792
Nbre d'acheteurs : Precision    12440
dtype: int64
Nbre de ref_achetées : in104    10479
dtype: int64
Précision moyenne:Precision    4.350402
dtype: float64


,customer_id,article_id,in104,inBaseline0
0,00039306476aaf41a07fed942884f16b30abfa83a2a8be...,624486001,1,0
1,000ed4cabda96069b547ae531c66de5f1bf6a7e5b83aa8...,894140005,1,0
2,000ed4cabda96069b547ae531c66de5f1bf6a7e5b83aa8...,894140005,1,0
3,000ed4cabda96069b547ae531c66de5f1bf6a7e5b83aa8...,882757004,1,0
4,000ed4cabda96069b547ae531c66de5f1bf6a7e5b83aa8...,827411001,1,0
...,...,...,...,...
54114,fff67e0ce3800aab23d16104d8ae13224f80053000dd6e...,805000008,1,0
54115,fff67e0ce3800aab23d16104d8ae13224f80053000dd6e...,769748014,1,0
54116,fff67e0ce3800aab23d16104d8ae13224f80053000dd6e...,868823007,1,0
54117,fff67e0ce3800aab23d16104d8ae13224f80053000dd6e...,915529005,1,0


In [46]:
Baseline_0(S7,article_history,'104')

Statistiques concernant le segment:
Taille du segment : 84991
Nbre d'acheteurs : Precision    9781
dtype: int64
Nbre de ref_achetées : in104    10000
dtype: int64
Précision moyenne:Precision    4.57213
dtype: float64


,customer_id,article_id,in104,inBaseline0
0,000493dd9fc463df1acc2081450c9e75ef8e87d5dd17ed...,399136061,1,0
1,000493dd9fc463df1acc2081450c9e75ef8e87d5dd17ed...,732842014,1,0
2,000493dd9fc463df1acc2081450c9e75ef8e87d5dd17ed...,556255001,1,0
3,000493dd9fc463df1acc2081450c9e75ef8e87d5dd17ed...,852219003,1,0
4,000493dd9fc463df1acc2081450c9e75ef8e87d5dd17ed...,732842021,1,0
...,...,...,...,...
44715,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,905365002,1,0
44716,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,863646005,1,0
44717,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,748269009,1,0
44718,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,881919001,1,0


In [47]:
Baseline_0(S8,article_history,'104')

Statistiques concernant le segment:
Taille du segment : 150520
Nbre d'acheteurs : Precision    16028
dtype: int64
Nbre de ref_achetées : in104    13159
dtype: int64
Précision moyenne:Precision    4.658223
dtype: float64


,customer_id,article_id,in104,inBaseline0
0,0003e867a930d0d6842f923d6ba7c9b77aba33fe2a0fbf...,913367001,1,0
1,0003e867a930d0d6842f923d6ba7c9b77aba33fe2a0fbf...,904026001,1,0
2,0003e867a930d0d6842f923d6ba7c9b77aba33fe2a0fbf...,905914002,1,0
3,0003e867a930d0d6842f923d6ba7c9b77aba33fe2a0fbf...,889036004,1,0
4,000abc5e9575b453ff1cde1e5d644f1c671daef57e0f85...,752657001,1,0
...,...,...,...,...
74657,fffaa422f09d92decce1eb732ea9a37c87ab8f951887b4...,904260001,1,0
74658,fffaa422f09d92decce1eb732ea9a37c87ab8f951887b4...,839332002,1,0
74659,fffaa422f09d92decce1eb732ea9a37c87ab8f951887b4...,921226002,1,0
74660,fffaa422f09d92decce1eb732ea9a37c87ab8f951887b4...,751471039,1,0


In [48]:
Baseline_0(S9,article_history,'104')

Statistiques concernant le segment:
Taille du segment : 83548
Nbre d'acheteurs : Precision    7571
dtype: int64
Nbre de ref_achetées : in104    10027
dtype: int64
Précision moyenne:Precision    4.549861
dtype: float64


,customer_id,article_id,in104,inBaseline0
0,0006d3ff0caf0cb4d4e0615ee5cb7d268622364d483335...,915529001,1,0
1,0006d3ff0caf0cb4d4e0615ee5cb7d268622364d483335...,930829001,1,0
2,0008804a45e7fbc8653ba8f5ce15880cb966ca220c52d2...,857271001,1,0
3,0008804a45e7fbc8653ba8f5ce15880cb966ca220c52d2...,857271001,1,0
4,000fab9caf56e8060680e5fd82d709ade7496318832201...,874937001,1,0
...,...,...,...,...
34442,ffef8aec5cf011fa1393b40337a5993ce0b7b81af6b322...,868320001,1,0
34443,ffef8aec5cf011fa1393b40337a5993ce0b7b81af6b322...,840909001,1,0
34444,fff69a9ce866cbde1b749376d30136edf14454a8a58cd0...,746300013,1,0
34445,fff69a9ce866cbde1b749376d30136edf14454a8a58cd0...,746300013,1,0


In [49]:
Baseline_0(S10,article_history,'104')

Statistiques concernant le segment:
Taille du segment : 85493
Nbre d'acheteurs : Precision    7999
dtype: int64
Nbre de ref_achetées : in104    9307
dtype: int64
Précision moyenne:Precision    4.196525
dtype: float64


,customer_id,article_id,in104,inBaseline0
0,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,777148006,1,0
1,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,835801001,1,0
2,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,923134005,1,0
3,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,865929003,1,1
4,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,572998013,1,0
...,...,...,...,...
33563,ffda862bfb72ef3b39c874a6d647cbdbb830e2e6a0bcc5...,842054002,1,0
33564,fffcc686584b3388a3afe410e3555e3557f556382b2f60...,817067002,1,0
33565,fffcc686584b3388a3afe410e3555e3557f556382b2f60...,840604001,1,0
33566,fffcc686584b3388a3afe410e3555e3557f556382b2f60...,723469001,1,0


In [50]:
Baseline_0(S11,article_history,'104')

Statistiques concernant le segment:
Taille du segment : 118625
Nbre d'acheteurs : Precision    12492
dtype: int64
Nbre de ref_achetées : in104    10481
dtype: int64
Précision moyenne:Precision    4.019613
dtype: float64


,customer_id,article_id,in104,inBaseline0
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,568601043,1,0
1,000172a9c322560c849754ffbdfdb2180d408aa7176b94...,685814001,1,0
2,000172a9c322560c849754ffbdfdb2180d408aa7176b94...,685814001,1,0
3,000172a9c322560c849754ffbdfdb2180d408aa7176b94...,685814001,1,0
4,0006bb0fad5c49341bd9cece264271e68e01a4e55f22ec...,708138021,1,0
...,...,...,...,...
50208,fff887d056fff68679d6e2491588ee4e83e57bf077ca7a...,903049004,1,0
50209,fff887d056fff68679d6e2491588ee4e83e57bf077ca7a...,896064004,1,0
50210,fffb287f12aea1204e9eabd5e02eaf7f3ed5f9abecd9a4...,821395005,1,0
50211,fffd870c6324ad3bda24e4d6aeae221c199479086bfdfd...,750423010,1,0


In [51]:
Baseline_0(S12,article_history,'104')

Statistiques concernant le segment:
Taille du segment : 135326
Nbre d'acheteurs : Precision    14442
dtype: int64
Nbre de ref_achetées : in104    10310
dtype: int64
Précision moyenne:Precision    3.764853
dtype: float64


,customer_id,article_id,in104,inBaseline0
0,00040239317e877c77ac6e79df42eb2633ad38fcac09fc...,875272011,1,0
1,00040239317e877c77ac6e79df42eb2633ad38fcac09fc...,875272012,1,0
2,0008968c0d451dbc5a9968da03196fe20051965edde741...,893059004,1,0
3,0008f66f5804877333ab9942c880b4a222f5b1a9f6ce27...,574109042,1,0
4,0008f66f5804877333ab9942c880b4a222f5b1a9f6ce27...,797079002,1,0
...,...,...,...,...
54367,fffa7d7799eb390a76308454cbdd76e473d65b1497fbe4...,861803014,1,0
54368,fffa7d7799eb390a76308454cbdd76e473d65b1497fbe4...,849886010,1,0
54369,fffb6dda3fec0b5c58aae22a19c039e0087da6625b54aa...,513512004,1,0
54370,fffbe76d05cba1b9bdf14bf67e58e7b73d4c2e3418f13c...,800691007,1,0


In [52]:
Baseline_0(S13,article_history,'104')

Statistiques concernant le segment:
Taille du segment : 90916
Nbre d'acheteurs : Precision    9349
dtype: int64
Nbre de ref_achetées : in104    8395
dtype: int64
Précision moyenne:Precision    3.614397
dtype: float64


,customer_id,article_id,in104,inBaseline0
0,0002cca4cc68601e894ab62839428e5f0696417fe0f9e8...,910601002,1,1
1,000b31552d3785c79833262bbeefa484cbc43d7b612b3c...,820613001,1,0
2,000b31552d3785c79833262bbeefa484cbc43d7b612b3c...,902224002,1,0
3,000b31552d3785c79833262bbeefa484cbc43d7b612b3c...,862693002,1,0
4,0015f16aa2702e2ec13d2e38052f496b9b915d3c64e82c...,865929007,1,0
...,...,...,...,...
33786,fff40baeae2bc423a2a90774386468a8afa147b6edce65...,767577058,1,0
33787,fff40baeae2bc423a2a90774386468a8afa147b6edce65...,891899004,1,0
33788,fff71b3986a9bc72e5f388d0e0c2390c96bb78023856d5...,896152002,1,1
33789,fff71b3986a9bc72e5f388d0e0c2390c96bb78023856d5...,574109039,1,0


In [53]:
Baseline_0(S14,article_history,'104')

Statistiques concernant le segment:
Taille du segment : 91750
Nbre d'acheteurs : Precision    8667
dtype: int64
Nbre de ref_achetées : in104    7294
dtype: int64
Précision moyenne:Precision    3.158879
dtype: float64


,customer_id,article_id,in104,inBaseline0
0,0005ed68483efa39644c45185550a82dd09acb07622acb...,863646004,1,0
1,0028cc9bb420077c0ad6739d17f80ca2ade0c3eebad796...,926500001,1,0
2,0028cc9bb420077c0ad6739d17f80ca2ade0c3eebad796...,893059005,1,0
3,002e0b61a6fbc3425d76f3f222750f60165d44e2572267...,801673002,1,0
4,002e0b61a6fbc3425d76f3f222750f60165d44e2572267...,801673002,1,0
...,...,...,...,...
27373,fff98edc27fc5d64c3027bf0e3702510143d1a79c3dc9a...,913688001,1,0
27374,fff98edc27fc5d64c3027bf0e3702510143d1a79c3dc9a...,874465001,1,0
27375,fff98edc27fc5d64c3027bf0e3702510143d1a79c3dc9a...,898918007,1,0
27376,fff98edc27fc5d64c3027bf0e3702510143d1a79c3dc9a...,929980001,1,0


In [64]:
### Modifications de la baseline0 avec les bonnes semaines en params

def Baseline_0 (segment,transact_history,X_week,y_week): # exemple: week104name ='in104' dans l'historique des transactions
    
    X_week_dummy="in" + f'{X_week}'
    y_week_dummy="in" + f'{y_week}'
    
    filtre_Xweek=transact_history[X_week_dummy]==1
    filtre_yweek=transact_history[y_week_dummy]==1
        
    X_achats=segment[['customer_id']].merge(transact_history[filtre_Xweek][['customer_id','article_id',X_week_dummy]])
    y_achats=segment[['customer_id']].merge(transact_history[filtre_yweek][['customer_id','article_id',y_week_dummy]]) 
    
    # calcul des plus populaires sur la dernière semaine des X
    articles= X_achats.groupby(['article_id']).sum()
    Top12_arr=articles.sort_values(by=X_week_dummy, ascending=False).head(12)
    Top12=""
    for i in range (0, 12):
        if i==0 :
            Top12=str(Top12_arr.index[i])
        else: 
            Top12=Top12 + " "+ str(Top12_arr.index[i])
            
    #je donne à tous mes customers de mon segment le baseline top12 calculée ci-dessus
    segment['Baseline0']=Top12
    
    #je retourne sur leurs achats et je vérifie si l'articleID acheté en semaine Y est dans la TOP12 de la semaine X (1 si oui, O sinon)
    y_achats['inBaseline0']=y_achats['article_id'].apply(lambda x: 1 if str(x) in Top12 else 0)
    Baseline0_precision=y_achats.groupby(['customer_id']).count().drop(columns= ['article_id', y_week_dummy]).rename(columns={'inBaseline0':'Precision'})
   
    print("Statistiques concernant le segment:")
    print("Taille du segment : " + f'{len(segment)}')
    print("Nbre d'acheteurs : " + f'{Baseline0_precision.count()}')
    print("Nbre de ref_achetées : " + f'{articles.count()}')
    print("Précision moyenne:" + f'{Baseline0_precision.mean()}')
    
    segment_BS=segment.merge(Baseline0_precision, how = 'left', on='customer_id') 
    #return segment_BS
    return "fin de traitement"

In [65]:
Baseline_0(S1,article_history,103,104)

Statistiques concernant le segment:
Taille du segment : 9553
Nbre d'acheteurs : Precision    1213
dtype: int64
Nbre de ref_achetées : in103    597
dtype: int64
Précision moyenne:Precision    2.656224
dtype: float64


'fin de traitement'

In [66]:
Baseline_0(S2,article_history,103,104)

Statistiques concernant le segment:
Taille du segment : 62030
Nbre d'acheteurs : Precision    6069
dtype: int64
Nbre de ref_achetées : in103    2309
dtype: int64
Précision moyenne:Precision    2.788103
dtype: float64


'fin de traitement'

In [67]:
Baseline_0(S3,article_history,103,104)

Statistiques concernant le segment:
Taille du segment : 122726
Nbre d'acheteurs : Precision    13203
dtype: int64
Nbre de ref_achetées : in103    4052
dtype: int64
Précision moyenne:Precision    3.102401
dtype: float64


'fin de traitement'

In [68]:
Baseline_0(S4,article_history,103,104)

Statistiques concernant le segment:
Taille du segment : 106736
Nbre d'acheteurs : Precision    13091
dtype: int64
Nbre de ref_achetées : in103    4660
dtype: int64
Précision moyenne:Precision    3.688794
dtype: float64


'fin de traitement'

In [69]:
Baseline_0(S5,article_history,103,104)

Statistiques concernant le segment:
Taille du segment : 111113
Nbre d'acheteurs : Precision    14092
dtype: int64
Nbre de ref_achetées : in103    5216
dtype: int64
Précision moyenne:Precision    4.037184
dtype: float64


'fin de traitement'

In [70]:
Baseline_0(S6,article_history,103,104)

Statistiques concernant le segment:
Taille du segment : 102792
Nbre d'acheteurs : Precision    12440
dtype: int64
Nbre de ref_achetées : in103    5198
dtype: int64
Précision moyenne:Precision    4.350402
dtype: float64


'fin de traitement'

In [71]:
Baseline_0(S7,article_history,103,104)

Statistiques concernant le segment:
Taille du segment : 84991
Nbre d'acheteurs : Precision    9781
dtype: int64
Nbre de ref_achetées : in103    4874
dtype: int64
Précision moyenne:Precision    4.57213
dtype: float64


'fin de traitement'

In [72]:
Baseline_0(S8,article_history,103,104)

Statistiques concernant le segment:
Taille du segment : 150520
Nbre d'acheteurs : Precision    16028
dtype: int64
Nbre de ref_achetées : in103    6794
dtype: int64
Précision moyenne:Precision    4.658223
dtype: float64


'fin de traitement'

In [73]:
Baseline_0(S9,article_history,103,104)

Statistiques concernant le segment:
Taille du segment : 83548
Nbre d'acheteurs : Precision    7571
dtype: int64
Nbre de ref_achetées : in103    4381
dtype: int64
Précision moyenne:Precision    4.549861
dtype: float64


'fin de traitement'

In [74]:
Baseline_0(S10,article_history,103,104)

Statistiques concernant le segment:
Taille du segment : 85493
Nbre d'acheteurs : Precision    7999
dtype: int64
Nbre de ref_achetées : in103    4190
dtype: int64
Précision moyenne:Precision    4.196525
dtype: float64


'fin de traitement'

In [75]:
Baseline_0(S11,article_history,103,104)

Statistiques concernant le segment:
Taille du segment : 118625
Nbre d'acheteurs : Precision    12492
dtype: int64
Nbre de ref_achetées : in103    5014
dtype: int64
Précision moyenne:Precision    4.019613
dtype: float64


'fin de traitement'

In [76]:
Baseline_0(S12,article_history,103,104)

Statistiques concernant le segment:
Taille du segment : 135326
Nbre d'acheteurs : Precision    14442
dtype: int64
Nbre de ref_achetées : in103    5178
dtype: int64
Précision moyenne:Precision    3.764853
dtype: float64


'fin de traitement'

In [77]:
Baseline_0(S13,article_history,103,104)

Statistiques concernant le segment:
Taille du segment : 90916
Nbre d'acheteurs : Precision    9349
dtype: int64
Nbre de ref_achetées : in103    3786
dtype: int64
Précision moyenne:Precision    3.614397
dtype: float64


'fin de traitement'

In [78]:
Baseline_0(S14,article_history,103,104)

Statistiques concernant le segment:
Taille du segment : 91750
Nbre d'acheteurs : Precision    8667
dtype: int64
Nbre de ref_achetées : in103    3256
dtype: int64
Précision moyenne:Precision    3.158879
dtype: float64


'fin de traitement'